In [ ]:
library(rliger)

library(Matrix)
library(Seurat)
library(spacexr)
library(Matrix)
library(doParallel)

In [ ]:
#Read in gene expression matrix and create seurat object 

gene_count <- readMM("/../genecount.mtx")
df_cell <- read.csv("/../df_cell.csv")
df_gene <- read.csv("/../df_gene.csv")
colnames(gene_count) = df_cell$Cell_name #your cell name column
rownames(gene_count) = df_gene$Gene_name
r = CreateSeuratObject(gene_count, meta.data = df_cell, project = "SpatialData")
r <- subset(r, subset = nCount_RNA >= 400 )







In [ ]:
#break down by region
cortex <- subset(r, subset = region == "cortex")
striatum <- subset(r, subset = region == "striatum")
hippocampus <- subset(r, subset = region == "hippocampus")


In [ ]:
#convert to liger object
striatum_liger = seuratToLiger(striatum, combined.seurat = TRUE, raw.assay = "RNA", meta.var = "source_dataset")
cortex_liger = seuratToLiger(cortex, combined.seurat = TRUE, raw.assay = "RNA", meta.var = "source_dataset")
hippocampus_liger = seuratToLiger(hippocampus, combined.seurat = TRUE, raw.assay = "RNA", meta.var = "source_dataset")


In [ ]:
#select 2000 var genes
cortex_liger = selectGenes(cortex_liger, num.genes = 2000 , do.plot = T)
striatum_liger = selectGenes(striatum_liger, num.genes = 2000 , do.plot = T)
hippocampus_liger = selectGenes(hippocampus_liger, num.genes = 2000 , do.plot = T)


In [ ]:
print(paste('A genes:', length(cortex_liger@var.genes)))
print(paste('A genes:', length(striatum_liger@var.genes)))
print(paste('A genes:', length(hippocampus_liger@var.genes)))

In [ ]:
#run LIGER 
cortex_liger <- scaleNotCenter(cortex_liger)
striatum_liger <- scaleNotCenter(striatum_liger)
hippocampus_liger <- scaleNotCenter(hippocampus_liger)



lambda_val = 5
knn_k_val = 35



cortex_liger <- optimizeALS(cortex_liger, k=28, thresh = 5e-5, nrep = 1)
cortex_liger <- quantileAlignSNF(cortex_liger, resolution = .8, knn_k = knn_k_val,small.clust.thresh = 30)

striatum_liger <- optimizeALS(striatum_liger, k=32, thresh = 5e-5, nrep = 1)
striatum_liger <- quantileAlignSNF(striatum_liger, resolution = .8, knn_k = knn_k_val,small.clust.thresh = 30)


hippocampus_liger <- optimizeALS(hippocampus_liger, k=32, thresh = 5e-5, nrep = 1)
hippocampus_liger <- quantileAlignSNF(hippocampus_liger, resolution = .8, knn_k = knn_k_val,small.clust.thresh = 30)


In [ ]:
cortex_liger <- optimizeALS(cortex_liger, k=12, thresh = 5e-5, nrep = 1)
cortex_liger <- quantileAlignSNF(cortex_liger, resolution = .8, knn_k = knn_k_val,small.clust.thresh = 30)

...


In [ ]:
cortex_liger = runUMAP(cortex_liger, min_dist = 0.1,  n_neighbors = 20)


In [ ]:
cortex_liger = louvainCluster(
  cortex_liger,
  )


In [ ]:
p_a_tsne1 = plotByDatasetAndCluster(cortex_liger , pt.size = 0.05, return.plots = T, raster = FALSE)
p_a_tsne1

In [ ]:
cortex_liger_Seurat = ligerToSeurat(cortex_liger)


In [ ]:
# Stash cell identity classes in metadata
cortex_liger_Seurat[["liger_clusters"]] <- Idents(object = cortex_liger_Seurat)
cortex_liger_Seurat <- StashIdent(object = cortex_liger_Seurat, save.name = "liger_clusters")


In [ ]:
DimPlot(cortex_liger_Seurat)

...


In [ ]:
#Iteratively remove low quality clusters that don't align spatially to tissue, and/or have very low UMI counts, and no
#distinct gene patterns 